## RunnableBranch: 입력에 따라 동적으로 로직 라우팅하기

### 입력에 따른 동적 로직 라우팅

라우팅을 수행하는 방법에는 두 가지가 있습니다.

1. RunnableLambda 에서 조건부로 실행 가능한 객체를 반환 (권장)
2. RunnableBranch 사용

### 간단한 예시

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """주어진 사용자 질문을 `수학`, `과학`, 또는 `기타` 중 하나로 분류하세요. 한 단어 이상으로 응답하지 마세요.

<question>
{question}
</question>

Classification:"""
)

# 체인을 생성합니다.
chain = (
    prompt
    | ChatOpenAI(model="gpt-4-turbo-preview")
    | StrOutputParser()  # 문자열 출력 파서를 사용합니다.
)

In [4]:
# 질문을 입력하여 체인을 호출합니다.
chain.invoke({"question": "2+2 는 무엇인가요?"})

'수학'

In [5]:
# 질문을 입력하여 체인을 호출합니다.
chain.invoke({"question": "작용 반작용의 법칙은 무엇인가요?"})

'과학'

In [6]:
# 질문을 입력하여 체인을 호출합니다.
chain.invoke({"question": "Google은 어떤 회사인가요?"})

'기타'

In [7]:
math_chain = (
    PromptTemplate.from_template(
        """You are an expert in math. \
Always answer questions starting with "깨봉선생님께서 말씀하시기를..". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    # OpenAI의 LLM을 사용합니다.
    | ChatOpenAI(model="gpt-4-turbo-preview")
)

science_chain = (
    PromptTemplate.from_template(
        """You are an expert in science. \
Always answer questions starting with "아이작 뉴턴 선생님께서 말씀하시기를..". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    # OpenAI의 LLM을 사용합니다.
    | ChatOpenAI(model="gpt-4-turbo-preview")
)

general_chain = (
    PromptTemplate.from_template(
        """Respond to the following question concisely:

Question: {question}
Answer:"""
    )
    # OpenAI의 LLM을 사용합니다.
    | ChatOpenAI(model="gpt-4-turbo-preview")
)

### 사용자 정의 함수 사용하기 (권장)

In [8]:
def route(info):
    # 주제에 "수학"이 포함되어 있는 경우
    if "수학" in info["topic"].lower():
        # datascience_chain을 반환
        return math_chain
    # 주제에 "과학"이 포함되어 있는 경우
    elif "과학" in info["topic"].lower():
        # art_chain을 반환
        return science_chain
    # 그 외의 경우
    else:
        # general_chain을 반환
        return general_chain

In [9]:
from langchain_core.runnables import RunnableLambda

full_chain = (
    {"topic": chain, "question": lambda x: x["question"]}
    | RunnableLambda(
        # 경로를 지정하는 함수를 인자로 전달합니다.
        route
    )
    | StrOutputParser()
)

In [10]:
# 수학과 관련된 질문을 입력하여 체인을 호출합니다.
full_chain.invoke({"question": "미적분의 개념에 대해 말씀해 주세요."})

'깨봉선생님께서 말씀하시기를, 미적분은 크게 두 가지 주요 개념인 미분과 적분으로 나뉩니다. 미분은 변화율을 다루는 개념으로, 어떤 함수의 순간적인 변화율을 구하는 과정을 말합니다. 즉, 어떤 곡선의 특정 지점에서의 기울기를 찾는 것과 같습니다. 반면, 적분은 주어진 구간에서의 면적, 부피 등을 계산하는 과정으로, 분할한 작은 부분들의 합을 구함으로써 전체의 값을 추정합니다. 미적분은 과학, 공학, 경제학 등 다양한 분야에서 중요한 역할을 하며, 복잡한 문제들을 해결하는 데 필수적인 도구입니다.'

In [11]:
# 과학과 관련된 질문을 입력하여 체인을 호출합니다.
full_chain.invoke({"question": "중력은 어떻게 작용하나요?"})

'아이작 뉴턴 선생님께서 말씀하시기를, 중력은 모든 질량을 가진 물체들 사이에 작용하는 힘이며, 이 물체들을 서로 끌어당기는 힘입니다. 이러한 중력의 크기는 두 물체의 질량에 비례하고, 물체들 사이의 거리의 제곱에 반비례합니다. 즉, 질량이 큰 물체일수록, 또는 거리가 가까울수록 중력은 더 강해집니다. 이 원리는 뉴턴의 만유인력 법칙으로 설명됩니다. 지구와 우리 사이에 작용하는 중력은 우리가 지면에 붙어 있게 하는 힘이며, 지구와 달, 태양과 지구 등 천체들 사이의 움직임을 설명하는 데에도 이 법칙이 적용됩니다.'

In [12]:
# 기타 질문을 입력하여 체인을 호출합니다.
full_chain.invoke({"question": "RAG(Retrieval Augmented Generation)은 무엇인가요?"})

'RAG(Retrieval Augmented Generation)는 검색 기반의 정보를 활용하여 보다 정확하고 관련성 높은 텍스트를 생성하는 자연어 처리 기술입니다. 기존의 데이터베이스나 문서 집합에서 관련 정보를 검색한 후, 이를 기반으로 새로운 텍스트를 생성하는 과정을 포함합니다. 이를 통해 단순한 패턴 인식을 넘어서 보다 정확하고 풍부한 지식 기반의 답변 생성이 가능해집니다.'

### RunnableBranch 사용하기

문법

- RunnableBranch는 (조건, Runnable) 쌍의 리스트와 기본 Runnable로 초기화됩니다.
- 호출 시 전달된 입력값을 각 조건에 전달하여 분기를 선택합니다.
- True로 평가되는 첫 번째 조건을 선택하고, 해당 조건에 해당하는 Runnable을 입력값과 함께 실행합니다.
- 제공된 조건과 일치하는 것이 없으면 기본 Runnable을 실행합니다.

In [13]:
from langchain_core.runnables import RunnableBranch

branch = RunnableBranch(
    # 주제에 "수학"이 포함되어 있는지 확인하고, 포함되어 있다면 math_chain을 실행합니다.
    (lambda x: "수학" in x["topic"].lower(), math_chain),
    # 주제에 "과학"이 포함되어 있는지 확인하고, 포함되어 있다면 science_chain을 실행합니다.
    (lambda x: "과학" in x["topic"].lower(), science_chain),
    # 위의 조건에 해당하지 않는 경우 general_chain을 실행합니다.
    general_chain,
)
# 주제와 질문을 입력받아 branch를 실행하는 전체 체인을 정의합니다.
full_chain = {"topic": chain,
              "question": lambda x: x["question"]} | branch | StrOutputParser()

In [14]:
# 질문을 입력하여 전체 체인을 실행합니다.
full_chain.invoke({"question": "미적분의 개념에 대해 말씀해 주세요."})

'깨봉선생님께서 말씀하시기를, 미적분은 크게 두 가지의 핵심적인 아이디어, 즉 미분과 적분으로 구성되어 있다고 합니다. \n\n미분은 주어진 함수의 순간적인 변화율을 측정하는 방법으로, 예를 들어 어떤 차의 속도는 시간에 따른 위치의 변화율을 미분하여 구할 수 있습니다. 이는 함수의 기울기, 즉 순간적인 변화를 나타내고, 이를 통해 접선의 기울기나 최대, 최소 문제를 해결할 수 있습니다.\n\n반면, 적분은 분할하여 합하는 과정을 통해 주어진 함수 아래의 면적을 찾는 방법입니다. 적분은 미분과 반대의 개념으로, 미분으로 나눠진 변화량들을 다시 합쳐 전체의 변화량을 찾는 것과 유사합니다. 이를 통해 속도가 주어졌을 때 거리를 구하거나, 함수가 나타내는 곡선 아래의 면적을 구할 수 있습니다.\n\n미적분은 물리학, 공학, 경제학 등 다양한 분야에서 중요한 역할을 하며, 복잡한 문제들을 해결하는 데 있어 필수적인 도구입니다. 따라서, 미적분의 개념을 이해하는 것은 고급 수학뿐만 아니라 다양한 실생활 문제를 해결하는 데도 큰 도움이 됩니다.'

In [15]:
# 질문을 입력하여 전체 체인을 실행합니다.
full_chain.invoke({"question": "중력 가속도는 어떻게 계산하나요?"})

'아이작 뉴턴 선생님께서 말씀하시기를, 중력 가속도는 지구 표면 근처에서 물체가 자유낙하할 때 받는 가속도로, 일반적으로 \\(9.8m/s^2\\)로 계산합니다. 이 값은 뉴턴의 만유인력 법칙에 의해 결정되며, 지구의 질량과 물체와 지구 중심 간의 거리에 따라 달라질 수 있습니다. 정확한 계산을 위해서는 \\(g = G \\frac{M}{r^2}\\) 공식을 사용할 수 있습니다. 여기서 \\(G\\)는 만유인력 상수, \\(M\\)은 지구의 질량, \\(r\\)은 지구 중심으로부터의 거리를 의미합니다. 하지만 일상생활에서는 대부분의 경우 \\(9.8m/s^2\\)를 사용하여 충분히 정확한 결과를 얻을 수 있습니다.'

In [16]:
# 질문을 입력하여 전체 체인을 실행합니다.
full_chain.invoke({"question": "RAG(Retrieval Augmented Generation)은 무엇인가요?"})

'RAG(Retrieval Augmented Generation)는 자연어 처리 분야에서 사용되는 기술로, 대규모 데이터베이스나 문서 집합에서 관련 정보를 검색(retrieval)하여 그 정보를 바탕으로 새로운 텍스트를 생성(generation)하는 모델입니다. 이 기술은 특히 질의 응답 시스템, 챗봇, 콘텐츠 추천 등에 활용되어, 보다 정확하고 유용한 답변을 생성하는 데 도움을 줍니다.'